In [1]:
!git clone https://github.com/maria-natale/ProgettoFVAB

Cloning into 'ProgettoFVAB'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 152 (delta 48), reused 85 (delta 21), pack-reused 0
Receiving objects: 100% (152/152), 163.85 KiB | 1.26 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!python "/content/ProgettoFVAB/Code/create_dataset.py"

file_dataset/spagnolo_giapponese.csv
djs
100% 6335/6335 [00:00<00:00, 635607.02it/s]
100% 18200/18200 [00:00<00:00, 563317.88it/s]


In [8]:
import os
import pandas as pd
import numpy as np
from keras.preprocessing import image
from tqdm import tqdm
from PIL import Image

path_git = '/content/ProgettoFVAB'
filename = 'spagnolo_giapponese'
path_drive = '/content/drive/MyDrive/Casillo&Natale'
dataset_dir = 'dataset_Spagnolo_giapponese'
LANGUAGES = {4:'Spagnolo',
  7: 'Giapponese'}


In [9]:
train = pd.read_csv(os.path.join(path_drive, dataset_dir+'/etichette', filename+'ImmaginiClasse_train.csv'))
train.head()
test = pd.read_csv(os.path.join(path_drive, dataset_dir+'/etichette', filename+'ImmaginiClasse_test.csv'))

Found 0 images belonging to 0 classes.


In [ ]:

# creating an empty list
train_image = []
path_images_train = os.path.join(path_drive, dataset_dir, 'train')

# for loop to read and store frames
#for i in tqdm(range(train.shape[0])):
for i in tqdm(range(80)):
  try:
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(path_images_train+'/'+train['frame'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
  except:
    print("corrotta")
    
# converting the list to numpy array
x_train = np.array(train_image)

# shape of the array
x_train.shape



 20%|██        | 14114/70350 [01:15<05:12, 179.68it/s]

corrotta


 21%|██        | 14579/70350 [01:18<05:23, 172.18it/s]

corrotta
corrotta


 21%|██        | 14654/70350 [01:18<05:28, 169.70it/s]

corrotta


 28%|██▊       | 19753/70350 [03:04<103:33:47,  7.37s/it]

In [ ]:
# creating an empty list
test_image = []
path_images_test = os.path.join(path_drive, dataset_dir, 'test')

# for loop to read and store frames
#for i in tqdm(range(test.shape[0])):
for i in tqdm(range(20)):
  try:
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(path_images_test+'/'+test['frame'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    test_image.append(img)
  except:
    print("corrotta")
    
# converting the list to numpy array
x_test = np.array(test_image)

# shape of the array
x_test.shape

In [ ]:
#y_train = train['language']
#y_test = test['language']
y_train = train.loc[:80, 1]
y_test = test.loc[:20, 1]

In [ ]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)


#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
#Reshape the data into 3-D array
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# Initialising the RNN
model = Sequential()
model.add(Bidirectional(LSTM(units = 100, return_sequences = True, input_shape = (x_train.shape[0], 1))))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(Bidirectional(LSTM(units = 100, return_sequences = True)))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(Bidirectional(LSTM(units = 100)))
model.add(Dropout(0.2))



# Adding the output layer
# For Full connection layer we use dense
model.add(Dense(8, activation='softmax'))

In [ ]:
import tensorflow as tf
#compile and fit the model on 30 epochs
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)



In [ ]:
model.fit(x_train, y_train, epochs = 2, batch_size = 50)
model.summary()

In [ ]:
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
model.evaluate(x_test, y_test)